In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import torch
import torch.nn as nn
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

In [4]:
pyro.set_rng_seed(0)

In [3]:
df = pd.read_excel('pack_benchmark_spec_v1.042.xlsm','Cells',header=[0,1],skiprows=1,nrows=1102)

## Data analysis

In [15]:
feature_set_0 = [('Capacity','Ah (nom)'),('Voltage','Nom'),
                 ('Mass','kg'),('Format','Unnamed: 13_level_1'),
                 ('Chemistry','Family'),('Chemistry','Cathode'),('Chemistry','Anode'),
                 ('Discharge','Ampsmax'),('Discharge','Ampscont'),
                 ('Charge','Ampsmax'),('Charge','Ampscont'),
                 ('Cycle Ageing','to 70% SoH'),('Cycle Ageing','to 80% SoH')]

In [ ]:
df.loc[:,feature_set_0].describe()

Capacity      Voltage        Mass     Discharge               \
          Ah (nom)          Nom          kg       Ampsmax     Ampscont   
count  1096.000000  1040.000000  996.000000    263.000000   749.000000   
mean     49.028663     3.468543    1.139255    306.046601   122.202017   
std      86.742960     0.521512    2.542226    717.768219   250.885797   
min       0.026000     1.200000    0.004200      2.500000     0.020000   
25%       3.432645     3.200000    0.064500     20.000000    10.000000   
50%       8.500000     3.600000    0.245000    128.000000    30.000000   
75%      57.250000     3.700000    1.245000    400.000000   150.000000   
max    1000.000000    12.000000   38.000000  10000.000000  3000.000000   

            Charge              Cycle Ageing                
           Ampsmax    Ampscont    to 70% SoH    to 80% SoH  
count   439.000000  674.000000     83.000000    180.000000  
mean     99.171392   36.199128   3084.939759   3306.966667  
std     259.607692   74.335621   3168.477889   5087.746627  
min       0.825000    0.008000    200.000000    250.000000  
25%       4.800000    1.770000    600.000000   1000.000000  
50%      17.500000    4.000000   2000.000000   2000.000000  
75%     110.000000   26.637500   5000.000000   3500.000000  
max    3000.000000  500.000000  13000.000000  35000.000000

In [21]:
df.loc[:,feature_set_0].describe(include=['object','category'])

Format Chemistry                              
       Unnamed: 13_level_1    Family                 Cathode Anode
count                 1094       783                     440   130
unique                   7        24                      19    17
top            Cylindrical       LFP  Lithium Iron Phosphate    Gr
freq                   461       340                     211    47

In [24]:
df.loc[:,'Format'].value_counts()

Unnamed: 13_level_1
Cylindrical            461
Prismatic              316
Pouch                  280
Blade                   19
Cylinridcal              8
cylindrical              8
Cylindrical              2
Name: count, dtype: int64

In [25]:
df.loc[:,('Chemistry','Family')].value_counts()

(Chemistry, Family)
LFP                    340
NMC                    200
NCA                     56
NMC, NCA, NMC/NCA       49
LCO                     41
LTO                     28
NaB                     17
NCM/NCA                 13
LMO                      7
Solid State              5
NiCd                     4
PbA                      4
NMCA                     3
NiMh                     3
LiS                      2
LMFP                     2
NMX                      2
BMLMP                    1
NMC 811                  1
Spinel                   1
Silicon carbon cell      1
LNCAO                    1
Li-S                     1
Chemistry                1
Name: count, dtype: int64

In [26]:
df.loc[:,('Chemistry','Cathode')].value_counts()

(Chemistry, Cathode)
Lithium Iron Phosphate    211
Nickel rich               154
Lithium Cobalt Oxide       33
NMC811                     11
NMC622                      7
Li metal/Liquid             5
Lithium Titanate            4
Gr+Si                       3
NaMO2                       2
NMC523                      1
NMC                         1
712                         1
NMC712                      1
NMC111+NCA                  1
High Nickel NCM+ Gr         1
High Nickel + SiC/SiOx      1
NMC955                      1
#                           1
40                          1
Name: count, dtype: int64

In [27]:
df.loc[:,('Chemistry','Anode')].value_counts()

(Chemistry, Anode)
Gr                  47
Graphite            28
Aluminium           10
Lithium Titanate     9
Si                   9
Li                   8
Gr/SiO               5
Gr-Si                4
SiOx                 2
Graphite+Silicon     1
Gr+LTO               1
Gr+SiO               1
Gr+Si                1
SiC                  1
Hard carbon          1
LTO                  1
Soft carbon          1
Name: count, dtype: int64

## ML model
We will use the VAE-based model from [this paper](https://www.sciencedirect.com/science/article/pii/S0031320320303046#bib0045). The basic idea is to use a neural network model to 'reconstruct' the data. This is done by forcing it through a bottleneck in the NN, the _latent space_.

In [ ]:
# Code currently borrowed directly from https://pyro.ai/examples/vae.html

class Decoder(nn.Module):
    def __init__(self, z_dim, hidden_dim, output_dim):
        super().__init__()
        # setup the two linear transformations used
        self.fc1 = nn.Linear(z_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, output_dim)
        # setup the non-linearities
        self.softplus = nn.Softplus()
        self.sigmoid = nn.Sigmoid()

    def forward(self, z):
        # define the forward computation on the latent z
        # first compute the hidden units
        hidden = self.softplus(self.fc1(z))
        # return the parameter for the output Bernoulli
        # each is of size batch_size x input_dim
        loc_img = self.sigmoid(self.fc21(hidden))
        return loc_img

In [6]:
class Encoder(nn.Module):
    def __init__(self, z_dim, hidden_dim, input_dim):
        super().__init__()
        # setup the three linear transformations used
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, z_dim)
        self.fc22 = nn.Linear(hidden_dim, z_dim)
        # setup the non-linearities
        self.softplus = nn.Softplus()

    def forward(self, x):
        # define the forward computation on the image x
        # first shape the mini-batch to have pixels in the rightmost dimension
        x = x.reshape(-1, self.input_dim)
        # then compute the hidden units
        hidden = self.softplus(self.fc1(x))
        # then return a mean vector and a (positive) square root covariance
        # each of size batch_size x z_dim
        z_loc = self.fc21(hidden)
        z_scale = torch.exp(self.fc22(hidden))
        return z_loc, z_scale